## Importação das libs utilizadas

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.stattools import adfuller

## Importação das bases

In [ ]:
PATH = '../input/walmart-recruiting-store-sales-forecasting/'

stores = pd.read_csv(PATH + 'stores.csv')
features = pd.read_csv(PATH + 'features.csv.zip', compression='zip')
train = pd.read_csv(PATH + 'train.csv.zip', compression='zip')
test = pd.read_csv(PATH + 'test.csv.zip', compression='zip')

## Funções úteis 

O bloco abaixo apresenta algumas funções úteis que foram utilizadas no decorrer do código.

In [ ]:
def check_missing_values(df, value_percent=False):
    '''
    Função responsável por verificar
    quantidade de missing values em 
    um dataframe.
    Entrada: Dataframe.
    Retorno: Dataframe.
    '''
    aux = df.isnull().sum().to_frame(name='qtde_missing')
    if value_percent:
        aux['perc_qtde_missing'] = aux['qtde_missing'].apply(lambda x: x/df.shape[0])
    return aux



def train_test_times_series(df, size_train):
    '''
    Função responsável por dividir uma base
    em treino/teste respeitando a order da
    serie.
    Entrada: df->Dataframe, size_train->porcentagem da base de treino
    Retorno: Dataframe
    '''
    
    tamanho_base = len(df)
    n_train = np.round(tamanho_base*size_train)
    
    treino = df.loc[0:n_train,:]
    teste = df.loc[(n_train+1):tamanho_base,:]
    return treino, teste


## Explorando as bases de dados

### Base de dados: Stores

Nessa parte o objetivo é conhecer um pouco mais das lojas e responder algumas perguntas como:

* Qual é a dimensão da base Store?
* Alguma das variáveis que compõem a base Stores contém valores ausentes?
* Quantas lojas são do Tipo A, B e C?
* Em média quais são os tamanhos das lojas? Tenho alguma com tamanho negativo?

In [ ]:
print('Qual é a dimensão da base Store?')
print('Temos {row} linhas e {col} colunas!'.format(row=stores.shape[0], col=stores.shape[1]))

In [ ]:
print('Alguma das variáveis que compõem a base Stores contém valores ausentes?')
check_missing_values(df=stores, value_percent=True)

In [ ]:
print('Quantas lojas são do tipo A, B e C ?')
qtde_lojas_tipo = stores.groupby('Type', as_index=False)['Store'].count()
qtde_lojas_tipo.rename(columns={"Store": "Qtde_Stores"}, inplace=True)
qtde_lojas_tipo.head()

In [ ]:
print('Em média quais são os tamanhos das lojas? Tenho alguma com tamanho negativo?')
stores['Size'].describe().to_frame()

### Base de dados: Features

Nessa parte o objetivo é conhecer um pouco mais das lojas e responder algumas perguntas como:

* Qual é a dimensão da base Fetures?
* Alguma das variáveis que compõem a base Features contém valores ausentes?
* Qual é o resumo estatístico de algumas variáveis?


In [ ]:
print('Qual é a dimensão da base Fetures?')
print('Temos {row} linhas e {col} colunas!'.format(row=features.shape[0], col=features.shape[1]))

In [ ]:
print('Alguma das variáveis que compõem a base Features contém valores ausentes?')
print('Pelo dataframe abaixo podemos peceber que mais de 50% da variável MarkDown 1-5 é representada por valores faltantes.')
check_missing_values(df=features, value_percent=True)

In [ ]:
print('Qual é o resumo estatístico de algumas variáveis?')
features.iloc[:,2:].describe()

### Base de dados: Train

Nessa parte o objetivo é conhecer um pouco mais dos dados de train.

* Qual é a dimensão da base Treino?
* Alguma das variáveis que compõem a base Features contém valores ausentes?
* Temos alguma venda negativa?
* Quantos departamentos temos por loja?

In [ ]:
print('Qual é a dimensão da base train?')
print('Temos {row} linhas e {col} colunas!'.format(row=train.shape[0], col=train.shape[1]))

In [ ]:
print('Alguma das variáveis que compõem a base train contém valores ausentes?')
check_missing_values(df=train, value_percent=True)

In [ ]:
print('Temos alguma venda negativa?')
print('Pela dataframe abaixo, podemos peceber que existe vendas semanais com valores negativos.')
train.iloc[:,3:].describe()

In [ ]:
print('Quantos departamentos temos por loja?')
train.groupby('Store')['Dept'].nunique().to_frame().head()

### Base de dados: Test

Nessa parte o objetivo é conhecer um pouco mais dos dados de test.

* Qual é a dimensão da base Test?
* Alguma das variáveis que compõem a base Features contém valores ausentes?

In [ ]:
print('Qual é a dimensão da base test?')
print('Temos {row} linhas e {col} colunas!'.format(row=test.shape[0], col=test.shape[1]))

In [ ]:
print('Alguma das variáveis que compõem a base test contém valores ausentes?')
check_missing_values(df=test, value_percent=True)

## Construção da base ABT (Analytical Base Table)

Após a pequena exploração nos dados, o próximo passo é a construção da nossa ABT. Isto é, a base que será usada para criação do modelo.

Nessa parte iremos realizar algumas ações sobre os seguintes pontos:

* Junção das bases
* Valores missing na base Features
* Valores negativos de vendas na base Train
* Criação de novas variáveis e algumas transformações
* Correlação entre as variáveis
* Normalização dos dados
* Verificação de estacionariedade nas séries

### Junção das bases

In [ ]:
train_full = train.merge(features, on=['Store', 'Date'], how='inner').merge(stores, on=['Store'], how='inner')
train_full.drop(columns=['IsHoliday_y'], inplace=True)
train_full.rename(columns={'IsHoliday_x':'IsHoliday'}, inplace=True)

print('Temos {row} linhas e {col} colunas!'.format(row=train_full.shape[0], col=train_full.shape[1]))

In [ ]:
test_full = test.merge(features, on=['Store', 'Date'], how='inner').merge(stores, on=['Store'], how='inner')
test_full.drop(columns=['IsHoliday_y'], inplace=True)
test_full.rename(columns={'IsHoliday_x':'IsHoliday'}, inplace=True)

print('Temos {row} linhas e {col} colunas!'.format(row=test_full.shape[0], col=test_full.shape[1]))

### Valores missing oriundos da base Features

Como o percentual de valores missing nas variáveis MarkDown 1-5 são altos, não será realizado nenhum tratamento nelas, uma vez que elas serão desconsideras da base em um primeiro momento.

Já as variáveis CPI e Unemployment utilizarei a mediana (agrupando por loja, pois cada loja percente a uma região) para imputação de dados.

In [ ]:
print('Removendo as variáveis MarkDown 1-5 da base train_abt e test_abt')
train_full.drop(columns=['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4','MarkDown5'], inplace=True)
test_full.drop(columns=['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4','MarkDown5'], inplace=True)

In [ ]:
print('Aplicando a mediana como imputação de dados para as variáveis: CPI e Unemployment ')
test_full['CPI'] = test_full['CPI'].fillna(test_full.groupby('Store')['CPI'].transform('median'))
test_full['Unemployment'] = test_full['Unemployment'].fillna(test_full.groupby('Store')['Unemployment'].transform('median'))

In [ ]:
print('Verificando novamente se temos valores ausentes na base train')
check_missing_values(df=train_full, value_percent=True)

In [ ]:
print('Verificando novamente se temos valores ausentes na base test')
check_missing_values(df=test_full, value_percent=True)

### Valores negativos de vendas na base Train

Para tratar o problema de vendas semanais negativas, resolvi trabalhar com faixas de intevalos. Sei que essas vendas não podem ser negativas, então o intervalo para a variável Weekly_Sales seria de 0 até a maior venda.

Como descrito na documentação do numpy: Dado um intervalo, os valores fora do intervalo são cortados nas bordas do intervalo. Por exemplo, se um intervalo de for especificado ([0,1]), valores menores que 0 se tornarão 0 e valores maiores que 1 se tornarão 1.

https://numpy.org/doc/stable/reference/generated/numpy.clip.html

In [ ]:
print('Aplicando tratamento de valores negativos na base')
train_full['Weekly_Sales'] = train_full.Weekly_Sales.clip(0, train_full.Weekly_Sales.max())

É importante ressaltar que substituir por zero as vendas negativas não é uma boa prática. Uma outra ação (que demandariam mais tempo) que poderia ser aplicada é: devsenvolver um modelo de ML para realizar uma inputação de dados.

### Criação de novas variáveis e alguns transformações

Nessa parte, será criado uma variável a partir de outra variável já existente, e transformaremos algumas variáveis categoricas em númericas. 

* Criaremos uma variável referente ao dia da semana, a partir da variável Date.

Segundo a documentação do pandas: Segunda-feira = 0, domingo = 6.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.dayofweek.html

* Transformação das variáveis IsHoliday e Type em númericas

In [ ]:
print('Transformando a variável Date em datetime')
train_full['Date'] = pd.to_datetime(train_full['Date'])
test_full ['Date'] = pd.to_datetime(test_full['Date'])

print('Criação da variavel dia da semana')

train_full['day_of_week'] = train_full['Date'].dt.dayofweek
test_full['day_of_week'] = test_full['Date'].dt.dayofweek

In [ ]:
print('Transformação das variáveis IsHoliday e Type em númericas')

train_full = pd.get_dummies(train_full, columns=['Type'])
train_full['IsHoliday'] = train_full['IsHoliday'].astype(int)


test_full = pd.get_dummies(test_full, columns=['Type'])
test_full['IsHoliday'] = test_full['IsHoliday'].astype(int)

### Correlação entre as variáveis

Apresentaremos uma matriz de correlação para verificar se existe alguma correlação entre as variáveis da base de treino.
Essa etapa é de grande valia, uma vez que usar variáveis altamente correlacionados acaba não fazendo sentido, pois teremos uma ou mais variáveis fornecendo as mesmas informações quando inseridos em um modelo de previsão. 

Sendo assim, é importante identificar essa correlação e se necessário remover algumas variáveis.

* Interpretação da correlação

 * Um valor próximo de +1 indica: um alto grau de associação entre as duas variáveis. Isto é, a medida que uma variável cresce a outra cresce também.
 
 * Um valor próximo de -1 indica: um alto grau de associação entre as duas variáveis. Isto é, a medida que uma variável cresce a outra decresce.
 
 * Um valor próximo a 0 indica: uma fraca ou ausente associação entre as variáveis.
 
* Para o cálculo da correção utilizarei o método de Sperman (não parámetrico), uma vez que o método usual de Pearson tem como pressuposto normalidade dos dados. 

In [ ]:
# Filtrando apenas as variáveis quantitativas
variaveis_corr = ['Weekly_Sales', 'Temperature','Fuel_Price', 'CPI', 'Unemployment', 'Size']
correlacao_train_full = train_full[variaveis_corr].corr(method = 'spearman')
plt.figure(figsize=(15, 10))
sns.heatmap(correlacao_train_full, annot=True)
plt.plot()

A princípio nenhuma variável precisará ser removida.

### Normalização dos dados

Será aplicado a normalização em algumas variáveis da base para corrigir problemas de escalas diferentes e também para uma melhor performance do modelo.

In [ ]:
colunas_trans_scaler = ['Temperature','Fuel_Price', 'CPI', 'Unemployment', 'Size']
trans_scaler = StandardScaler().fit(train_full[colunas_trans_scaler])

train_full[colunas_trans_scaler] = trans_scaler.transform(train_full[colunas_trans_scaler])
test_full[colunas_trans_scaler] = trans_scaler.transform(test_full[colunas_trans_scaler])

### Verificação de estacionariedade nas séries

Formalmente dizendo, uma série é definida como estacionária quando ela se desenvolve no tempo aleatoriamente ao redor de uma média constante. (Veja http://www.portalaction.com.br/series-temporais/11-estacionariedade#:~:text=Uma%20s%C3%A9rie%20temporal%20%C3%A9%20dita,estacionariedade%2C%20por%20exemplo%2C%20tend%C3%AAncia.)
De maneira resumida, as as propriedas estatísticas da série não mudam conforme o tempo.

Isso é importante ser verificado, pois caso a série não seja estacionária e abordarmos um modelos de séries temporais (ex: ARIMA, SARIMAX) um dos pressupostos desses modelos é que a série seja estacionária.

Como também, se abordarmos modelos de Machine Leaning, alguns parâmetros como sazonalidade podem ser mais facilmente modelados, outros como a tendência podem ser mais dificeis para o modelo entender.

* Teste de Raíz unitária (teste de Dickey-Fuller)

Para verificação da estacionariedade das séries, utilizaremos o teste de Raíz unitária. O teste tem como hipótese nula que os dados não são estacionários. Considerando um nível de confiança de 95%, temos que:

 * Se o p-valor do teste for maior que 0.05 não rejeitamos a hipótese nula, isto é, a série é não estacionária.
 * Se o p-valor do teste for menor que 0.05 rejeitamos a hipótese nula, isto é, a série é estacionária.

In [ ]:
store_dept = []
for i in range(0,len(list(train_full['Store'].unique()))):
    for j in list(train_full.loc[train_full.Store == i, 'Dept'].unique()):
        aux = train_full.loc[(train_full.Store == i) & (train_full.Dept == j)]
        try:
            p_value = adfuller(aux.Weekly_Sales)[1]
            if p_value > 0.05:
                store_dept.append((i,j, p_value))
        except:
            continue

In [ ]:
# df_serie_nestac são as séries não estacionárias segundo teste da Raíz unitária 

df_serie_nestac = pd.DataFrame(store_dept, columns = ['store','dept','p_value'])
df_serie_nestac['chave'] = df_serie_nestac['store'].astype(str) + '_' + df_serie_nestac['dept'].astype(str)

In [ ]:
df_serie_nestac.head()

Pelo dataframe acima (df_serie_nestac), podemos ver que algumas séries não satisfazem a propriedade de estacionariedade.

Inicialmente vou modelar a série sem nenhuma tratativa para esse problema, a fim de verificarmos se o modelo de ML conseguirá modelar essas tendências (sazonais ou não).

Em um segundo momento, podemos testar dois modelos de ML dividindo as séries em dois grandes grupos: as que são estacionárias e as que não são estacionárias.

O meu objetivo ao propor esse "agrupamento" de séries, é colocar as séries que possuem as mesmas propriedades/se comportam de maneira parecida uma para cada modelo.

## Modelagem - Considerando todos as séries

Construindo nossa ABT

In [ ]:
# Não consideraremos Type_A,Type_B e Type_C, uma vez que eles já estão representados na variável Size

features_abt = ['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Temperature', 
                'Fuel_Price', 'CPI', 'Unemployment', 'Size', 'day_of_week']

train_abt = train_full[features_abt]

### Divisão da base em treino e teste para validação da qualidade do modelo

Um ponto importante quando trabalhamos com modelagem, é obtermos algumas medidas para verificarmos a precisão de um modelo, isto é, medidas que podem nos ajudar a verificar o quão bom é um modelo. 

Geralmente para obtermos esses tipos de medidas precisamos dos valores previstos pelo modelo e dos valores reais observados. Com base nisso, uma maneira de obtermos essas medidas é dividirmos a série de dados em bases de treinamento e teste, respeitando a ordem da série (para modelos temporais). Assim, o modelo pode ser construído no conjunto de dados de treinamento e as previsões podem ser feitas e avaliadas no conjunto de dados de teste.

Essa divisão pode ser feita selecionando um ponto de divisão arbitrário na série de observações e criando dois novos conjuntos de dados. Assim, se tivermos uma série de 100 observações podemos usar uma divisão do tipo: 70 a 30. Em que as 70 primeiras observações faram parte do conjunto de dados de treinamento e as 30 seguintes observações faram parte do conjunto de dados de teste. 

### Modelando

In [ ]:
train_abt_sort = train_abt.sort_values(by=['Store','Dept','Date']).reset_index(drop=True)

Para este problema, dividirei a base em 90% para treino do modelo e 10% para teste. Isto é, 90% dos primeiros valores da série ordenada será para treinamento e os 10% restantes será para teste.

In [ ]:
treino, teste = train_test_times_series(train_abt_sort, 0.90)

In [ ]:
print('Dimensão da base de treino é {shape}'.format(shape=treino.shape))

In [ ]:
print('Dimensão da base de teste é {shape}'.format(shape=teste.shape))

In [ ]:
features = ['Store', 'Dept', 'IsHoliday', 'Temperature',
           'Fuel_Price', 'CPI', 'Unemployment', 'Size', 'day_of_week']

target = 'Weekly_Sales'

In [ ]:
X_train, X_test = treino[features], teste[features]
y_train, y_test = treino[target], teste[target]

### Modelo considerado: ExtraTreesRegressor


Para a modelagem de uma série temporal, podemos encontrar na literatura modelos como:

* Suavização Exponencial
* SARIMA, SARIMAX 

Todavia, também podemos utilizar modelos regressores comuns (respeitando os conceitos de uma série temporal) como:

* Árvores de Decisão (RandomForest, ExtraTreesRegressor)
* Redes neurais simples (MLP)


In [ ]:
etr = ExtraTreesRegressor(n_estimators=100, random_state=42)
etr.fit(X_train, y_train)
print('R² da ExtraTreesRegressor na base teste foi: {0:.2f}'.format(etr.score(X_test, y_test)))

## Modelagem - Considerando dois grupos de séries

Construindo nossa ABT levando em considerando dois grandes grupos: séries estacionárias e séries não estacionárias

Nessa parte dividimos a base de treino em dois grandes grupos: as que são estacionárias (series_estac) e as que não são (series_n_estac).

In [ ]:
train_full['chave'] = train_full['Store'].astype(str) + '_' + train_full['Dept'].astype(str)

In [ ]:
series_estac = train_full.loc[~train_full.chave.isin(df_serie_nestac['chave'])]

series_n_estac = train_full.loc[train_full.chave.isin(df_serie_nestac['chave'])]

Construindo nossa ABT tanto para as séries estacionárias quanto para as não estacionárias

In [ ]:
features_abt = ['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Temperature', 
                'Fuel_Price', 'CPI', 'Unemployment', 'Size', 'day_of_week']

train_abt_est = series_estac[features_abt]
train_abt_n_est = series_n_estac[features_abt]

### Modelando as séries estacionárias

In [ ]:
train_abt_est_sort = train_abt_est.sort_values(by=['Store','Dept','Date']).reset_index(drop=True)

In [ ]:
treino_est, teste_est = train_test_times_series(train_abt_est_sort, 0.90)

In [ ]:
features = ['Store', 'Dept', 'IsHoliday', 'Temperature',
           'Fuel_Price', 'CPI', 'Unemployment', 'Size', 'day_of_week']

target = 'Weekly_Sales'

In [ ]:
X_train_est, X_test_est = treino_est[features], teste_est[features]
y_train_est, y_test_est = treino_est[target], teste_est[target]

In [ ]:
etr_est = ExtraTreesRegressor(n_estimators=100, random_state=42)
etr_est.fit(X_train_est, y_train_est)
print('R² da ExtraTreesRegressor na base teste foi: {0:.2f}'.format(etr_est.score(X_test_est, y_test_est)))

### Modelando as séries não estacionárias

In [ ]:
train_abt_nest_sort = train_abt_n_est.sort_values(by=['Store','Dept','Date']).reset_index(drop=True)

In [ ]:
# Aplicando a diferenciação de ordem 1

train_abt_nest_sort.loc[:,'diff_Weekly_Sales'] = train_abt_nest_sort.groupby(['Store','Dept'])['Weekly_Sales'].diff(1)

In [ ]:
train_abt_nest_sort.dropna(inplace=True)

In [ ]:
treino_nest, teste_nest = train_test_times_series(train_abt_nest_sort, 0.90)

In [ ]:
features = ['Store', 'Dept', 'IsHoliday', 'Temperature',
            'Fuel_Price', 'CPI', 'Unemployment', 'Size', 'day_of_week']

target = 'diff_Weekly_Sales'

In [ ]:
X_train_nest, X_test_nest = treino_nest[features], teste_nest[features]
y_train_nest, y_test_nest = treino_nest[target], teste_nest[target]

In [ ]:
etr_nest = ExtraTreesRegressor(n_estimators=100, random_state=42)

etr_nest.fit(X_train_nest, y_train_nest)
print('R² da ExtraTreesRegressor na base teste foi: {0:.2f}'.format(etr_nest.score(X_test_nest, y_test_nest)))

## Modelo final

Comparando os resultados obtidos, podemos perceber que a primeira versão de modelagem (Considerando todos as séries) apresentou um melhor desempenho (um R² de 0,86).

Podemos perceber também que o R² obtido na modelagem das séries não estacionárias foi bem baixo. Uma hipótese que surge, é que provavelmente a tendência presente nas séries pode ser explicada pelas variéveis.


Modelagem escolhida: Modelagem - Considerando todos as séries


Próximos passos:

* Treinar novamente o modelo considerando a base train completamente (sem divisão treino/teste)
* Aplicar o modelo na base test

In [ ]:
features = ['Store', 'Dept', 'IsHoliday', 'Temperature',
            'Fuel_Price', 'CPI', 'Unemployment', 'Size', 'day_of_week']

target = 'Weekly_Sales'

In [ ]:
X, y = train_abt_sort[features], train_abt_sort[target]

etr_model = ExtraTreesRegressor(n_estimators=100, random_state=42)
etr_model.fit(X, y)

In [ ]:
features_test_abt = ['Store', 'Dept', 'Date', 'IsHoliday', 'Temperature', 
                     'Fuel_Price', 'CPI', 'Unemployment', 'Size', 'day_of_week']

test_abt = test_full[features_test_abt]

In [ ]:
test_abt_sort = test_abt.sort_values(['Store','Dept','Date']).reset_index(drop=True)

In [ ]:
test_abt_sort['pred_ETR'] = etr_model.predict(test_abt_sort[features])

In [ ]:
test_abt_sort.shape

## Submission

In [ ]:
submission = pd.DataFrame()

In [ ]:
submission['Id'] = test_abt_sort['Store'].astype(str) + '_' + test_abt_sort['Dept'].astype(str) + '_' + test_abt_sort['Date'].astype(str)

In [ ]:
submission['Weekly_Sales'] = test_abt_sort['pred_ETR']

In [ ]:
submission.to_csv('submission.csv', index=False)